# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    print(root)
    print(dirs)
#ignore hidden .ipynb_checkpoints subdirecotry,so files within it won't be walked
    dirs[:] = [d for d in dirs if not d[0] == '.']
    print("new dirs:{}".format(dirs))
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)
    print(len(file_path_list))

/home/workspace
/home/workspace/event_data
['.ipynb_checkpoints']
new dirs:[]
['/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-1

In [17]:
# Alternative to get all files paths list
# files_list_test=glob.glob('/home/workspace/event_data/*.csv')
# print(files_list_test)
# len(files_list_test)

['/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/work

30

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 

# Loop over all files path
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:         
        csvreader = csv.reader(csvfile) 
        next(csvreader) ##escape header
        # looping each line of the file f     
        for line in csvreader:            
            full_data_rows_list.append(line)          


csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    # write the column name as first row in f
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    # wrire the rest of data into f
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra data modeling

## Now CSV file titled <font color=red>event_datafile_new.csv</font> is ready to work with, located within current working directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="image_event_datafile_new.jpg">

#### Connect to cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS myetl 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('myetl')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data






#### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
#### This query is asking for sesssionid and itemInSession, these two fields can uniquely define each row, so they are primary key.

In [8]:
# Create the data model based on the query
query1 ="""CREATE TABLE IF NOT EXISTS music_session(
            session_id int,
            item_in_session int,
            artist text,
            song text,
            length float, 
            PRIMARY KEY(session_id,item_in_session))"""
try:
    session.execute(query1)
except Exception as e:
    print(e)

In [9]:
# Load the data into created table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_session(session_id,item_in_session,artist,song,length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"        
        session.execute(query,(int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

This query returns artist name, song name, and the length of song when session_id=338 and item_in_session=4.

In [10]:
# Test the loaded data in table by querying database
select_test1=("SELECT artist,song,length FROM music_session \
              WHERE session_id=338 AND item_in_session=4")
rows=session.execute(select_test1)
for row in rows:
    print(row.artist,row.song,row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### The returned record which matched session_id=338 and item_in_session=4 is listed below:
Faithless Music Matters (Mark Knight Dub) 495.30731201171875

#### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
#### This query is asking for userid and sessionid, they are part of primary key, also item_in_session is clustering column uniquely identify each row in table.

In [11]:
# Create the data model based on the query
query2 ="""CREATE TABLE IF NOT EXISTS music_user(
            user_id int,
            session_id int, 
            item_in_session int,
            firstname text,
            lastname text,
            artist text,
            song text,             
            PRIMARY KEY((user_id,session_id),item_in_session))"""
try:
    session.execute(query2)
except Exception as e:
    print(e)
                    

In [12]:
# Load the data into created table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_user(user_id,session_id,item_in_session,firstname,lastname,artist,song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query,(int(line[10]),int(line[8]),int(line[3]),line[1],line[4],line[0],line[9]))

The following query returns all records which match the specific user_id=10 and session_id=182.

In [13]:
# Test the loaded data in table by querying database
select_test2=("SELECT item_in_session,artist,song,firstname,lastname FROM music_user \
              WHERE user_id=10 AND session_id=182")
rows=session.execute(select_test2)
for row in rows:
    print(row.artist,row.song,row.firstname,row.lastname,row.item_in_session,)

Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


#### There are four records returned when user_id=10 and session_id=182:
Down To The Bone Keep On Keepin' On Sylvie Cruz 0  
Three Drives Greece 2000 Sylvie Cruz 1  
Sebastien Tellier Kilometer Sylvie Cruz 2  
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3

#### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
#### This query is asking for song, so song is partition key. And it's also ask for user's name, user_id and song together are compostie key.

In [14]:
# Create the data model based on the query
query3 ="""CREATE TABLE IF NOT EXISTS music_song(
            song text,
            user_id int,
            firstname text,
            lastname text,             
            PRIMARY KEY(song,user_id))"""
try:
    session.execute(query3)
except Exception as e:
    print(e)

                    

In [15]:
# Load the data into created table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_song(song,user_id,firstname,lastname)"
        query = query + "VALUES (%s, %s, %s, %s)"     
        session.execute(query,(line[9],int(line[10]),line[1],line[4]))


The following query returns all people's first name and last name who listened the song 'All Hands Against His Own'.

In [16]:
# Test the loaded data in table by querying database
select_test3=("SELECT firstname,lastname FROM music_song \
              WHERE song='All Hands Against His Own'")
rows=session.execute(select_test3)
for row in rows:
    print(row.firstname,row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


#### There are three person who listened this song 'All Hands Against His Own'
Jacqueline Lynch  
Tegan Levine  
Sara Johnson

### Drop the tables before closing out the sessions

In [33]:
query_drop1=("DROP TABLE IF EXISTS music_session")
try:
    session.execute(query_drop1)
except Exception as e:
    print(e)
    
query_drop2=("DROP TABLE IF EXISTS music_user")
try:
    session.execute(query_drop2)
except Exception as e:
    print(e)

query_drop3=("DROP TABLE IF EXISTS music_song")
try:
    session.execute(query_drop3)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [34]:
session.shutdown()
cluster.shutdown()